In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

In [2]:
PIPELINE_NAME = "weather-type-pipeline"
SCHEMA_PIPELINE_NAME = "weather-type-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Tahap Pertama

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [6]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

interactive_context.show(statistics_gen.outputs["statistics"])

In [7]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Atmospheric_Pressure',FLOAT,required,,-
'Cloud_Cover',STRING,required,,'Cloud_Cover'
'Humidity',INT,required,,-
'Location',STRING,required,,'Location'
'Precipitation',INT,required,,-
'Season',STRING,required,,'Season'
'Temperature',INT,required,,-
'UV_Index',INT,required,,-
'Visibility',FLOAT,required,,-


,Values
Domain,
'Cloud_Cover',"'clear', 'cloudy', 'overcast', 'partly cloudy'"
'Location',"'coastal', 'inland', 'mountain'"
'Season',"'Autumn', 'Spring', 'Summer', 'Winter'"
'Weather_Type',"'Cloudy', 'Rainy', 'Snowy', 'Sunny'"


In [8]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

interactive_context.show(example_validator.outputs['anomalies'])

In [9]:
TRANSFORM_MODULE_FILE = "weather_transform.py"

In [10]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "Weather_Type"

def transformed_name(key):
    return key + '_xf'

def preprocessing_fn(inputs):
    outputs = {}

    # Normalize numeric features
    outputs[transformed_name('Temperature')] = tft.scale_to_z_score(inputs['Temperature'])
    outputs[transformed_name('Humidity')] = tft.scale_to_z_score(inputs['Humidity'])
    outputs[transformed_name('Wind_Speed')] = tft.scale_to_z_score(inputs['Wind_Speed'])
    outputs[transformed_name('Precipitation')] = tft.scale_to_z_score(inputs['Precipitation'])
    outputs[transformed_name('Atmospheric_Pressure')] = tft.scale_to_z_score(inputs['Atmospheric_Pressure'])
    outputs[transformed_name('UV_Index')] = tft.scale_to_z_score(inputs['UV_Index'])
    outputs[transformed_name('Visibility')] = tft.scale_to_z_score(inputs['Visibility'])

    # Encode categorical features
    outputs[transformed_name('Cloud_Cover')] = tft.compute_and_apply_vocabulary(inputs['Cloud_Cover'])
    outputs[transformed_name('Season')] = tft.compute_and_apply_vocabulary(inputs['Season'])
    outputs[transformed_name('Location')] = tft.compute_and_apply_vocabulary(inputs['Location'])

    # Compute vocabulary for the label and convert labels to integer indices
    weather_type_indices = tft.compute_and_apply_vocabulary(inputs[LABEL_KEY])
    weather_type_one_hot = tf.one_hot(weather_type_indices, depth=4)
    outputs[transformed_name(LABEL_KEY)] = tf.reshape(weather_type_one_hot, [-1, 4])

    return outputs


Overwriting weather_transform.py


In [11]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\weather-type-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\54ba0c2121e64da498aca25bdcb39e17\assets


INFO:tensorflow:Assets written to: pipelines\weather-type-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\54ba0c2121e64da498aca25bdcb39e17\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['metadata', 'transformed_metadata', 'transform_fn']

In [13]:
import pprint
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 5 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Atmospheric_Pressure_xf"
    value {
      float_list {
        value: 0.13295313715934753
      }
    }
  }
  feature {
    key: "Cloud_Cover_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "Humidity_xf"
    value {
      float_list {
        value: 0.21113531291484833
      }
    }
  }
  feature {
    key: "Location_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Precipitation_xf"
    value {
      float_list {
        value: 0.8812830448150635
      }
    }
  }
  feature {
    key: "Season_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Temperature_xf"
    value {
      float_list {
        value: -0.2961544096469879
      }
    }
  }
  feature {
    key: "UV_Index_xf"
    value {
      float_list {
        value: -0.5195206999778748
      }
    }
  }
  feature {
    key: "Visibility_xf"
    value {
      float_list {
   

In [14]:
TRAINER_MODULE_FILE = "weather_trainer.py"

In [15]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "Weather_Type"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    
    def format_data(features, labels):
        labels = tf.reshape(labels, [-1, 4])
        return features, labels

    return dataset.map(format_data)


def model_builder():
    """Build model"""
    # Input layer
    inputs = []
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Temperature')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Humidity')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Wind_Speed')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Precipitation')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Atmospheric_Pressure')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('UV_Index')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Visibility')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Cloud_Cover')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Season')))
    inputs.append(tf.keras.Input(shape=(1,), name=transformed_name('Location')))
    
    # Concatenate input layers
    x = tf.keras.layers.Concatenate()(inputs)
    
    # Hidden layers
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    
    # Output layer
    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
    
    # Create model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn

def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    return transformed_features

  return transform_features_fn

def run_fn(fn_args: FnArgs):
    """Train the model"""
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    # Get train & eval data
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=1)
    
    # Build model
    model = model_builder()
    
    # Train model
    model.fit(train_dataset, epochs=10, validation_data=eval_dataset)
    
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples')),
        'transform_features':
        _get_transform_features_signature(model, tf_transform_output),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting weather_trainer.py


In [16]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Epoch 1/10
1655/1655 [==============================] - 9s 5ms/step - loss: 0.3713 - categorical_accuracy: 0.8748 - val_loss: 0.2495 - val_categorical_accuracy: 0.9019
Epoch 2/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.2183 - categorical_accuracy: 0.9124 - val_loss: 0.2187 - val_categorical_accuracy: 0.9107
Epoch 3/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.1964 - categorical_accuracy: 0.9166 - val_loss: 0.2069 - val_categorical_accuracy: 0.9123
Epoch 4/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.1846 - categorical_accuracy: 0.9197 - val_loss: 0.2022 - val_categorical_accuracy: 0.9161
Epoch 5/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.1771 - categorical_accuracy: 0.9224 - val_loss: 0.1994 - val_categorical_accuracy: 0.9146
Epoch 6/10
1655/1655 [==============================] - 5s 3ms/step - loss: 0.1718 - categorical_accuracy: 0.9237 - val_loss: 0.1987 - val_categorical_accuracy:

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\weather-type-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\weather-type-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [27]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [44]:
import tensorflow_model_analysis as tfma 
_LABEL_KEY = 'Weather_Type_xf'
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=_LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives'),
                tfma.MetricConfig(
                    class_name='CategoricalAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.9}
                        ),
                    )
                ),
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
    ])

In [45]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 22
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [46]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'categorical…

In [47]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
from tfx.types import standard_artifacts

# Function to check model blessing
def is_model_blessed(blessing):
    return blessing.get_int_custom_property('blessed') == 1

# Run Evaluator and check blessing status
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=eval_config
)

interactive_context.run(evaluator)

# Check if the model is blessed
model_blessing_artifact = evaluator.outputs['blessing'].get()[0]

if is_model_blessed(model_blessing_artifact):
    pusher = Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory='serving_model_dir/weather_classification_model'))
    )
    
    interactive_context.run(pusher)
else:
    print("Model not blessed, not pushing the model.")
